In [17]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point

#1 - UFO Sightings data
ufo_sightings = pd.read_csv("Resources/scrubbed.csv")

#2 - Clean UFO Sightings data
ufo_clean = ufo_sightings.loc[ufo_sightings["country"]=="us"]
ufo_clean[['month', 'day', 'year']] = ufo_clean['datetime'].str.split('/',expand=True)
ufo_clean[['year', 'time']] = ufo_clean['year'].str.split(' ',expand=True)
ufo_clean=ufo_clean.dropna()
ufo_clean['duration (seconds)']=pd.to_numeric(ufo_clean['duration (seconds)'])
ufo_clean['latitude']=pd.to_numeric(ufo_clean['latitude'])
ufo_clean['longitude']=pd.to_numeric(ufo_clean['longitude ']) #note: 'longitude ' has a space at the end
ufo_clean['month']=pd.to_numeric(ufo_clean['month'])
ufo_clean['day']=pd.to_numeric(ufo_clean['day'])
ufo_clean['year']=pd.to_numeric(ufo_clean['year'])
#ufo_clean.dtypes

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
#3 - Convert UFO Sightings to Geopandas Geodataframe
gdf_sightings = GeoDataFrame( 
    ufo_clean.drop(['longitude ','latitude'],axis=1),
    crs={'init':'epsg:4326'}, #WGS84 coordinate system
    geometry=[Point(xy) for xy in zip(ufo_clean.longitude ,ufo_clean.latitude)])
#gdf_sightings = gpd.GeoDataFrame(ufo_clean,geometry=gpd.points_from_xy(ufo_clean.longitude ,ufo_clean.latitude))
gdf_sightings.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,month,day,year,time,longitude,geometry
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,10,10,1949,20:30,-97.941111,POINT (-97.94111 29.88306)
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,10,10,1956,21:00,-96.645833,POINT (-96.64583 28.97833)
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,10,10,1960,20:00,-157.803611,POINT (-157.80361 21.41806)
5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,10,10,1961,19:00,-82.188889,POINT (-82.18889 36.59500)
7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,10,10,1965,23:45,-73.408333,POINT (-73.40833 41.11750)


In [19]:
#4 - Military Bases data
military = gpd.read_file("Resources/military-bases.geojson")
#military.head()

In [20]:
#5 - Adds a buffer of 5 mile to UFO Sightings
ufo_buffer = gdf_sightings
#ufo_buffer = gdf_sightings.loc[gdf_sightings["year"]==2013]
#ufo_buffer = ufo_buffer.loc[ufo_buffer["month"]==7]
ufo_buffer.geometry = gdf_sightings.geometry.buffer(5*(1/60)) #Note: Rough conversion of degree to mile (1/60)
#ufo_buffer.head()

In [21]:
#6 - Spatial Join of UFO Sightings intersecting Military Bases using a buffer of 5 miles
ufo_join = gpd.sjoin(ufo_buffer,military,op="intersects")
ufo_join

,datetime,city,state,country_left,shape,duration (seconds),duration (hours/min),comments,date posted,month,...,shape_area,site_name,objectid,area,country_right,component,perimeter,joint_base,shape_leng,objectid_1
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,10,...,1.431264e+07,MCB Hawaii (Kaneohe Bay),1240,4.763906,United States,MC Active,13.490841,N/A,23474.767590,164
3976,10/22/2013 22:15,kaneohe,hi,us,oval,480.0,8 minutes,Oval bright red object with yellow flickering ...,11/11/2013,10,...,1.431264e+07,MCB Hawaii (Kaneohe Bay),1240,4.763906,United States,MC Active,13.490841,N/A,23474.767590,164
7625,1/10/2014 22:58,kaneohe,hi,us,fireball,90.0,90 seconds,A silent red ball of light traveled off the pa...,1/16/2014,1,...,1.431264e+07,MCB Hawaii (Kaneohe Bay),1240,4.763906,United States,MC Active,13.490841,N/A,23474.767590,164
14635,1/14/2014 20:50,waimanalo,hi,us,fireball,5.0,5 seconds,I saw an orange fireball with flames trailing ...,1/16/2014,1,...,1.431264e+07,MCB Hawaii (Kaneohe Bay),1240,4.763906,United States,MC Active,13.490841,N/A,23474.767590,164
14856,1/15/2004 11:00,kailua,hi,us,light,120.0,2 minutes,Two brightly reflective objects skimming only ...,1/31/2004,1,...,1.431264e+07,MCB Hawaii (Kaneohe Bay),1240,4.763906,United States,MC Active,13.490841,N/A,23474.767590,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75097,9/16/2003 20:15,elba,al,us,light,3600.0,1 hour +,light in sky changing course moving at rapid r...,9/17/2003,9,...,9.653376e+05,Stinson 5AB Stagefield AL,77,0.270940,United States,Army Active,2.098668,N/A,4058.937370,37
75097,9/16/2003 20:15,elba,al,us,light,3600.0,1 hour +,light in sky changing course moving at rapid r...,9/17/2003,9,...,1.458209e+06,Runkle Stagefield AL,74,0.409439,United States,Army Active,2.812786,N/A,5437.820888,34
75253,9/17/1999 20:50,anacoco,la,us,disk,2400.0,40 minutes,My wife called me to the living room to see an...,10/2/1999,9,...,1.332110e+09,Fort Polk,63,375.459637,United States,Army Active,332.773511,N/A,641870.160928,25
78091,9/27/2012 23:00,pace,fl,us,other,120.0,1-2 minutes,What I saw looked like a helicopter&#44 which ...,10/30/2012,9,...,3.476849e+06,NOLF Spencer,5558,0.990832,United States,Navy Active,3.892805,N/A,7455.613608,648
